* seq2seq 모델과 이를 위한 어텐션 메커니즘

### 1) Sequence-to-Sequence, seq2seq
* 입력된 시퀀스로부터 다른 도메인의 시퀀스를 출력하는 다양한 분야에서 사용되는 모델
* ex. 챗봇, 기계 번역
  * 챗봇: 입력 시퀀스와 출력 시퀀스를 각각 질문과 대답으로 구성
  * 번역기: 입력 시퀀스와 출력 시퀀스를 각각 입력 문장과 번역 문장으로 만들기
* 그 외에도 내용 요약(Text Summarization), STT(Speech to Text)

#### 1. Sequence-to-Sequence
* 번역기에서 대표적으로 사용되는 모델
* 크게 두 개로 구성된 아키텍처: 인코더, 디코더
  * 인코더: 입력 문장의 모든 단어들을 순차적으로 입력받은 뒤에 마지막에 이 모든 단어 정보들을 압축해서 하나의 벡터로 만듦. 이를 컨텍스트 벡터(context vector)
  * 디코터: 컨텍스트 벡터를 받아서 번역된 단어를 한 개씩 순차적으로 출력
  * 인코더와 디코더 아키텍처의 내부는 사실 두 개의 **RNN 아키텍처**
  * 입력 문장을 받는 RNN 셀을 인코더라고 하고, 출력 문장을 출력하는 RNN 셀을 디코더
  * 성능 문제로 실제로는 LSTM 셀 또는 GRU 셀로 구성됨
  * 인코더: 
  1. **입력 문장**은 단어 토큰화를 통해서 단어 단위로 쪼개지고, 단어 토큰은 각각은 RNN 셀의 각 시점의 입력이 됨. 
  2. **인코더 RNN 셀**은 모든 단어를 입력받은 뒤에 *인코더 RNN 셀의 마지막 시점의 은닉 상태*를 디코더 RNN 셀로 넘겨주는데, 이를 **컨텍스트 벡터**라고 함
  * 컨텍스트 벡터는 디코더 RNN 셀의 첫번째 은닉 상태로 사용됨
  * 디코더: 기본적으로 **RNNLM(RNN Language Model)**
    * 테스트 과정: <sos>가 입력되면, 다음에 등장할 확률이 높은 단어를 예측
      * 첫번째 시점의 디코더 RNN 셀은 예측된 단어 je를 다음 시점의 RNN 셀의 입력으로 입력
      * 끝을 의미하는 <eos>
    * <정리> 훈련 과정: 훈련 과정에서는 디코더에게 인코더가 보낸 컨텍스트 벡터와 **실제 정답인 상황인 <sos> je suis étudiant를 입력**받았을 때,  je suis étudiant <eos>가 나와야 된다고 정답을 알려주면서 훈련함. **교사 강요(teacher forcing)**


  * RNN 셀은 t-1에서의 은닉 상태와 t에서의 입력 벡터를 입력으로 받고, t에서의 은닉 상태를 만듦
  * 현재 시점에서 t에서의 은닉 상태: 과거 시점의 동일한 RNN 셀에서의 모든 은닉 상태의 값들의 영향을 누적해서 받아온 값
    * **컨텍스트 벡터**: 인코더에서의 마지막 RNN 셀의 은닉 상태값. 이는 입력 문장의 모든 단어 토큰들의 정보를 요약해서 담고 있음
  * **seq2seq 모델은 선택될 수 있는 모든 단어들로부터 하나의 단어를 골라서 예측해야됨**
    * 이를 위하여 **소프트맥스 함수** 사용
    * 디코더에서 각 시점의 RNN셀에서 출력 벡터가 나오면, 해당 벡터는 소프트맥스 함수를 통해 출력 시퀀스의 각 단어별 확률값으로 반환하고, 디코더는 출력 단어를 결정


### 2. 글자 레벨 기계 번역기(Character-Level Neural Machine Translation) 구현하기
* 기계 번역기를 훈련시키기 위해서는 훈련 데이터로 병렬 코퍼스(parallel corpus)가 필요. 이는 두 개 이상의 언어가 병렬적으로 구성된 코퍼스 의미


### 1) 병렬 코퍼스 데이터에 대한 이해와 전처리
* 태깅 단어의 병렬 데이터: 쌍이 되는 모든 데이터가 길이가 같았음
* 여기서는 쌍이 된다고 해서 길이가 같지 않음
* 토큰의 개수가 2인 문장을 넣었을 때 토큰 4개인 문장이 나올 수 있음
* seq2seq는 기본적으로 입력 시퀀스와 출력 시퀀스가 다를 수 있다고 가정

In [ ]:
import os
import shutil
import zipfile

import pandas as pd
import tensorflow as tf
import urllib3
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
http = urllib3.PoolManager()
url ='http://www.manythings.org/anki/fra-eng.zip'
filename = 'fra-eng.zip'
path = os.getcwd()
zipfilename = os.path.join(path, filename)
with http.request('GET', url, preload_content=False) as r, open(zipfilename, 'wb') as out_file:       
    shutil.copyfileobj(r, out_file)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
    zip_ref.extractall(path)

In [ ]:
lines = pd.read_csv('fra.txt', names=['src', 'tar', 'lic'], sep='\t')
del lines['lic']
len(lines)

190206

In [ ]:
lines = lines.loc[:, 'src':'tar']
lines = lines[0:60000] # 6만개만 저장
lines.sample(10) #랜덤 10개

,src,tar
53099,Can I continue my trip?,Puis-je poursuivre mon voyage ?
21158,What is going on?,Qu'y a-t-il ?
51670,We hear with our ears.,On entend par les oreilles.
37985,Tom wants a new car.,Tom veut une nouvelle voiture.
7499,I'm adaptable.,Je m'adapte.
36389,My back still hurts.,Mon dos me fait encore mal.
6835,He's after me.,Il est après mes fesses.
30063,It's a coincidence.,C'est une coïncidence.
14491,Is it all wrong?,Est-ce tout faux ?
21038,We're untalented.,Nous sommes dépourvus de talent.


In [ ]:
#  <sos>와 <eos> 대신 '\t'를 시작 심볼, '\n'을 종료 심볼로 간주하여 추가
lines.tar = lines.tar.apply(lambda x : '\t '+ x + ' \n')
lines.sample(10)

,src,tar
58268,Tom knows what you did.,\t Tom sait ce que tu as fait. \n
57249,Tennis is loads of fun.,\t Le tennis est extrêmement divertissant. \n
4226,You're rich.,\t Tu es riche. \n
52285,Why are you flinching?,\t Pourquoi tressailles-tu ? \n
54434,I believe this is mine.,\t Je crois que c'est le mien. \n
19990,The water is hot.,\t L'eau est brûlante. \n
33596,Do you drink coffee?,\t Buvez-vous du café ? \n
27543,Both girls laughed.,\t Les deux filles ont ri. \n
32740,You can't leave me.,\t Vous ne pouvez pas me quitter. \n
42313,I'm not taking sides.,\t Je ne vais pas prendre parti. \n


In [ ]:
# 글자 집합 구축
# 토큰 단위가 단어가 아닌 글자
src_vocab = set()
for line in lines.src: #1줄씩 읽음
  for char in line: #1개의 글자씩 읽음
    src_vocab.add(char)

tar_vocab = set()
for line in lines.tar:
  for char in line:
    tar_vocab.add(char)

In [ ]:
src_vocab_size = len(src_vocab)+1
tar_vocab_size = len(tar_vocab)+1
print(src_vocab_size)
print(tar_vocab_size)

79
105


In [ ]:
# 이 중에서 인덱스를 임의로 부여하여 일부만 출력
# 현 상태에서 인덱스를 사용하려고 하면 에러가 남. 하지만 정렬하여 순서를 정해준 뒤에 인덱스를 사용하여 출력하면 됨.
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))
print(src_vocab[45:75])
print(tar_vocab[45:75])

# 글자 집합에 글자 단위로 저장된 것을 확인

['W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
['T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w']


In [ ]:
# 각 글자에 인덱스를 부여
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
print(src_to_index)
print(tar_to_index)

{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, '&': 6, "'": 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, '?': 23, 'A': 24, 'B': 25, 'C': 26, 'D': 27, 'E': 28, 'F': 29, 'G': 30, 'H': 31, 'I': 32, 'J': 33, 'K': 34, 'L': 35, 'M': 36, 'N': 37, 'O': 38, 'P': 39, 'Q': 40, 'R': 41, 'S': 42, 'T': 43, 'U': 44, 'V': 45, 'W': 46, 'X': 47, 'Y': 48, 'Z': 49, 'a': 50, 'b': 51, 'c': 52, 'd': 53, 'e': 54, 'f': 55, 'g': 56, 'h': 57, 'i': 58, 'j': 59, 'k': 60, 'l': 61, 'm': 62, 'n': 63, 'o': 64, 'p': 65, 'q': 66, 'r': 67, 's': 68, 't': 69, 'u': 70, 'v': 71, 'w': 72, 'x': 73, 'y': 74, 'z': 75, 'é': 76, '’': 77, '€': 78}
{'\t': 1, '\n': 2, ' ': 3, '!': 4, '"': 5, '$': 6, '%': 7, '&': 8, "'": 9, '(': 10, ')': 11, ',': 12, '-': 13, '.': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20, '6': 21, '7': 22, '8': 23, '9': 24, ':': 25, '?': 26, 'A': 27, 'B': 28, 'C': 29, 'D': 30, 'E': 31, 'F': 32, 'G': 33, 'H': 34, 'I': 3

In [ ]:
# 인덱스가 부여된 글자 집합으로부터 갖고 있는 훈련 데이터에 정수 인코딩 수행
# 우선 인코더의 입력이 될 영어 문장 샘플에 대해서 정수 인코딩을 수행해보고, 5개의 샘플을 출력

encoder_input = []
for line in lines.src:
  temp_X = []
  for w in line: # 각 줄에서 1개씩 글자 읽음
    temp_X.append(src_to_index[w]) # 글자에 해당되는 정수로 변환
  encoder_input.append(temp_X)
print(encoder_input[:5])

[[30, 64, 10], [30, 64, 10], [30, 64, 10], [31, 58, 10], [31, 58, 10]]


In [ ]:
# 디코더의 입력이 될 프랑스어 데이터에 대해서 정수 인코딩 수행

decoder_input = []
for line in lines.tar:
  temp_X = []
  for w in line:
    temp_X.append(tar_to_index[w])
  decoder_input.append(temp_X)
print(decoder_input[:5])

[[1, 3, 48, 53, 3, 4, 3, 2], [1, 3, 39, 53, 70, 55, 60, 57, 14, 3, 2], [1, 3, 28, 67, 73, 59, 57, 3, 4, 3, 2], [1, 3, 45, 53, 64, 73, 72, 3, 4, 3, 2], [1, 3, 45, 53, 64, 73, 72, 14, 3, 2]]


In [ ]:
# 아직 정수 인코딩을 수행해야 할 데이터가 하나 더 남았음
# 디코더의 예측값과 비교하기 위한 '실제값'이 필요
# 실제값에는 시작 심볼에 해당하는 <sos>가 있을 필요가 없음

decoder_target = []
for line in lines.tar:
  t = 0
  temp_X = []
  for w in line:
    if t>0:
      temp_X.append(tar_to_index[w])
    t=t+1
  decoder_target.append(temp_X)
print(decoder_target[:5])

# 디코더의 입력값에 해당되는 decoder_input 데이터에 비교하면 decoder_input에서는 모든 문장의 앞에 붙어있던 숫자 1이
# decoder_target에서는 제거된 것을 볼 수 있음

[[3, 48, 53, 3, 4, 3, 2], [3, 39, 53, 70, 55, 60, 57, 14, 3, 2], [3, 28, 67, 73, 59, 57, 3, 4, 3, 2], [3, 45, 53, 64, 73, 72, 3, 4, 3, 2], [3, 45, 53, 64, 73, 72, 14, 3, 2]]


In [ ]:
# 모든 데이터에 대해서 정수 인덱스로 변경하였으니 패딩 작업 수행
# 패딩을 위해서 영어 문장과 프랑스어 문장 각각 가장 길이가 긴 샘플 길이 알아보기

max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])
print(max_src_len)
print(max_tar_len)

24
76


In [ ]:
# 영어는 영어끼리, 프랑스어는 프랑스어끼리 길이를 밪춤
encoder_input = pad_sequences(encoder_input, maxlen = max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen = max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen = max_tar_len, padding='post')

In [ ]:
# 모든 값에 대해서 원핫 인코딩 수행
# 글자 단위 번역기이므로 워드 임베딩은 별도로 사용되지 않으며, 
# 예측값과의 오차 측정에 사용되는 실제값 뿐만 아니라 입력값도 원핫 벡터를 사용

encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

#### 2) 교사 강요 (teacher Forcing)
* **현재 시점의 디코더 셀의 입력은 오직 이전 디코더 셀의 출력을 입력으로 받는데, decoder_input이 왜 필요할까?**
* 훈련 과정에서는 이전 시점의 **디코더 셀의 출력**을 현재 시점의 디코더 셀의 입력으로 넣어주지 않고, 이전 시점의 **실제값**을 현재 시점의 디코더 셀의 입력값으로 하는 방법 사용
* 이유: 이전 시점의 디코더 셀의 예측이 틀렸는데, 이를 현재 시점의 디코더 셀의 입력으로 사용하면 현재 시점의 디코더 셀의 예측도 잘못될 가능성이 높고, 이는 연쇄 작용으로 디코더 전체의 예측을 어렵게 함
  * 이런 상황이 반복되면 훈련 시간이 느려짐
* 이와 같이 RNN의 모든 시점에 대해서 이전 시점의 예측값 대신 실제값을 입력으로 주는 방법을 교사 강요라고 함


#### 3) seq2seq 기계 번역기 훈련시키기

In [7]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np

In [ ]:
encoder_inputs = Input(shape=(None, src_vocab_size))
encoder_lstm = LSTM(units=256, return_state=True) #LSTM의 은닉 상태 크기는 256. 인코더의 내부 상태를 디코더로 넘겨 줘야 하기에 return_state=True
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
# encoder_outputs도 같이 리턴받기는 했지만 여기서는 필요없으므로 이 값은 버림
encoder_states = [state_h, state_c]
# LSTM은 바닐라 RNN과는 달리 상태가 두 개. 바로 은닉 상태와 셀 상태

In [ ]:
# 은닉 상태와 셀 상태를 모두 디코더로 전달. 이것이 컨텍스트 벡터

decoder_inputs = Input(shape=(None, tar_vocab_size))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
# 디코더의 첫 상태를 인코더의 은닉 상태, 셀 상태로 합니다
decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# 디코더는 인코더의 마지막 은닉 상태를 초기 은닉 상태로 사용. initial_state=encoder_states가 이 의미
# 동일하게 디코더의 은닉 상태 크기도 256을 줌
# 디코더도 은닉 상태, 셀 상태를 리턴하기는 하지만 훈련 과정에서는 사용하지 않음
# 그 후 출력층에 프랑스어의 단어 집합의 크기만큼 뉴런을 배치한 후 소프트맥스 함수를 상요하여 실제값과의 오차를 구함

In [ ]:
model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=64, epochs=40, validation_split=0.2)

# 입력으로는 인코더 입력과 디코더 입력
# 디코더의 실제값인 decoder_target도 필요
# 배치 크기는 64, 총 40 에포크를 학습
# 위에서 설정한 은닉 상태의 크기와 에포크 수는 실제로 훈련 데이터에 과적합 상태를 불러옴
# 중간부터 검증 데이터에 대한 오차인 val_loss가 올라가는데, 일단 내버려 두기

#### 4) seq2seq 기계 번역기 동작시키기
1. 번역하고자 하는 입력 문장이 인코더에 들어가서 은닉 상태와 셀 상태를 얻습니다.
2. 상태와 <SOS>에 해당하는 '\t'를 디코더로 보냅니다.
3. 디코더가 <EOS>에 해당하는 '\n'이 나올 때까지 다음 문자를 예측하는 행동을 반복합니다.

In [ ]:
# 인코더 정의
# encoder_inputs와 encoder_states는 훈련 과정에서 이미 정의한 것 재사용
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [ ]:
#디코더 설계

# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(256, ))
decoder_state_input_c = Input(shape=(256, ))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_state_inputs)

# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 이용
# 이는 뒤의 함수 decode_sequence()에 구현
decoder_states = [state_h, state_c]

# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태인 state_h와 state_c를 버리지 않음
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

In [ ]:
# 단어로부터 인덱스를 얻는 것이 아니라 인덱스로부터 단어를 얻을 수 있는
# index_to_src와 index_to_tar를 만듦

index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

In [ ]:
def decode_sequence(input_seq):
  # 입력으로부터 인코더의 상태를 얻음
  state_value = encoder_model.predict(input_seq)

  # <SOS>에 해당하는 원핫 벡터 생성
  target_seq = np.zeros((1, 1, tar_vocab_size))
  target_seq[0, 0, tar_to_index['\t']] = 1

  stop_condition = False
  decoded_sentence = ""

  # stop_condition이 True가 될 때까지 루프 반복
  while not stop_condition:
    # 이점 시점의 상태 state_value를 현 시점의 초기 상태로 사용
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

    # 예측 결과를 문자로 변환
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = index_to_tar[sampled_token_index]

    # 현재 시점의 예측 문자를 예측 문장에 추가
    decoded_sentence += sampled_char

    # <eos>에 도달하거나 최대 길이를 넘으면 중단
    if (sampled_char == '\n' or len(decoded_sentence) > max_tar_len):
      stop_condition = True

    # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, sampled_token_index] = 1

    # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
    states_value = [h, c]

  return decoded_sentence

In [ ]:
for seq_index in [3, 50, 100, 300, 1001]: #입력 문장의 인덱스
  input_seq = encoder_input[seq_index: seq_index + 1]
  decoded_sentence = decode_sentence(input_seq)
  print(35 * "-")
  print('입력 문장:', lines.src[seq_index])
  print('정답 문장:', lines.tar[seq_index][1:len(lines.tar[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
  print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1]) # '\n'을 빼고 출력

### 2) Word-Level 번역기 만들기(Neural Machine Translation (seq2seq) Tutorial)
* 이번에는 word-level로 만들기. 모델 아키텍처는 이전 챕터와 동일하지만, 단어 레벨을 수행하는 만큼 추가적인 전처리와 임데잍 층, 그리고 추가적인 후처리 작업이 필요

In [1]:
# 데이터 로드 및 전처리

import os
import re
import shutil
import zipfile

import numpy as np
import pandas as pd
import tensorflow as tf
import unicodedata
import urllib3
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [2]:
num_samples = 33000

http = urllib3.PoolManager()
url = 'http://www.manythings.org/anki/fra-eng.zip'
filename = 'fra-eng.zip'
path = os.getcwd()
zipfilename = os.path.join(path, filename)
with http.request('GET', url, preload_content=False) as r, open(zipfilename, 'wb') as out_file:
    shutil.copyfileobj(r, out_file)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
    zip_ref.extractall(path)

In [3]:
# 전처리 함수들 구현
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
                 if unicodedtat.category(c) != 'Mn')
  
def preprocess_sentence(sent):
  # 위에서 구현한 함수를 내부적으로 호출
  sent = unicode_to_ascii(sent.lower())

  # 단어와 구두점 사이에 공백을 만듦
  # Ex) "he is a boy." => "he is a boy ."
  sent = re.sub((r"([?.!,¿])", r" \1", sent)

  # (a-z, A-Z, ".", "?", "!", ",") 이들을 제외하고는 전부 공백으로 변환합니다.
  sent = re.sub(r"[^a-zA-Z!.?]+", r" ", sent)

  sent = re.sub(r"\s+", " ", sent)
  return sent

SyntaxError: ignored

In [ ]:
# 구현한 전처리 함수들을 임의의 문장을 입력으로 테스트
# 전처리 테스트
en_sent = u"Have you had dinner?"
fr_sent = u"Avez-vous déjà diné?"
print(preprocess_sentence(en_sent))
print(prepocess_sentence(fr_sent).encode('utf-8'))

In [ ]:
# 훈련 과정에서 Teacher Forcing을 사용할 예정이므로
# 훈련 시 사용할 디코더의 입력 시퀀스와 실제 값에 해당하는 출력 시퀀스를 따로 분리하여 저장
# 입력 시퀀스에는 시작을 의미하는 토큰인 <sos> 추가
# 출력 시퀀스에는 종료를 의미하는 토큰인 <eos> 추가

def load_preprocessed_data():
  encoder_input, decoder_input, decoder_target = [], [], []

  with open("fra.txt", "r") as lines:
    for i, line in enumerate(lines):

      # source 데이터와 target 데이터 분리
      src_line, tar_line, _ = line.strip().split('\t')

      # source 데이터 전처리
      src_line_input = [w for w in preprocess_sentence(src_line).split()]

      # target 데이터 전처리
      tar_line = preprocess_sentence(tar_line)
      tar_line_input = [w for w in ("<sos> " + tar_line).split()]
      tar_line_target = [w for w in ("<sos> " + tar_line).split()]

      encoder_input.append(src_line_input)
      decoder_input.append(tar_line_input)
      decoder_target.append(tar_line_target)

      if i == num_samples - 1:
        break
  return encoder_input, decoder_input, decoder_target #인코더 입력, 디코더 입력, 디코더 실제값

In [ ]:
sents_en_in, sents_fra_in, sents_fra_out = load_preprocessed_data()
print(sents_en_in[:5])
print(sents_fra_in[:5])
print(sents_fra_out[:5])

In [ ]:
# 케라스 토크나이저를 통해 단어 집합을 생성하고, 텍스트 시퀀스를 정수 시퀀스로 변환하는 정수 인코딩 과정 거침

tokenizer_en = Tokenizer(filters="", lower=False)
tokenizer_en.fit_on_texts(sents_en_in)
encoder_input = tokenizer_en.texts_to_sequences(sents_en_in)

tokenizer_fra = Tokenizer(filters="", lower=False)
tokenizer_fra.fit_on_texts(sents_fra_in)
tokenizer_fra.fit_on_texts(sents_fra_out)
decoder_input = tokenizer_fra.texts_to_sequences(sents_fra_in)
decoder_target = tokenizer_fra.texts_to_sequences(sents_fra_out)

In [ ]:
# 패딩
encoder_input = pad_sequences(encoder_input, padding="post")
decoder_input = pad_sequences(decoder_input, padding="post")
decoder_target = pad_sequences(decoder_target, padding="post")

# 샘플은 총 33000개, 영어 문장 길이는 8, 프랑스어 문장 길이는 16

In [ ]:
# 단어 집합의 크기 정의
src_vocab_size = len(tokenizer_en.word_index) + 1
tar_vocab_size = len(tokenizer_fra.word_index) + 1
print("영어 단어 집합의 크기 : {:d}, 프랑스어 단어 집합의 크기 : {:d}".format(src_vocab_size, tar_vocab_size))

In [ ]:
# 단어로부터 정수를 얻는 딕셔너리와 정수로부터 단어를 얻는 딕셔너리를 각각 만들어줌
# 이들은 훈련을 마치고 예측 과정과 실제값과 결과를 비교하는 경우에 사용됨

src_to_index = tokenizer_en.word_index
index_to_src = tokenizer_en.index_word # 훈련 후 결과 비교할 때 사용

tar_to_index = tokenizer_fra.word_index # 훈련 후 예측 과정에서 사용
index_to_Tar = tokenizer_fra.index_word # 훈련 후 결과 비교시 사용

In [ ]:
# 테스트 데이터를 분리할 차례
# 분리 전, 적절한 분포를 갖도록 데이터를 섞어주는 과정 진행
# 이를 위해서 우선 순위가 섞인 정수 시퀀스 리스트를 만듦

indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
print(indices)

In [ ]:
# 임의로 출력
# decoder_input과 decoder_target은 데이터의 구조상으로 앞에 붙은 <sos> 토큰과 뒤에 붙은 <eos>를 제외하면
# 동일한 정수 시퀀스를 가져야 하므로 이를 확인해주면 됨

encoder_input[30997]

In [ ]:
decoder_input[30997]

In [ ]:
decoder_target[30997]

In [ ]:
# 훈련 데이터의 10%를 테스트 데이터로 분리

n_of_val = int(33000*0.1)
print(n_of_val)

In [ ]:
encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

In [ ]:
# 훈련 데이터와 테스트 데이터의 크기(shape)를 출력

print(encoder_input_train.shape)
print(decoder_input_train.shape)
print(decoder_target_train.shape)
print(encoder_input_test.shape)
print(decoder_input_test.shape)
print(decoder_target_test.shape)

### 2. 기계 번역기 만들기

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

In [ ]:
latent_dim = 50

In [ ]:
# 인코더 설계
# Masking은 패딩 토큰인 숫자 0의 경우에는 연산을 제외하는 역할 수행

encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(src_vocab_size, latent_dim)(encoder_inputs) #임베딩 층
enc_masking = Masking(mask_value=0.0)(enc_emb) #패딩 0은 연산에서 제외
encoder_lstm = LSTM(latent_dim, return_state=True) #상태값 리턴을 위해 return_state는 True
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking) #은닉 상태와 셀 상태를 리턴
encoder_states = [state_h, state_c] #인코더의 은닉 상태와 셀 상태를 저장

In [ ]:
# 디코더

decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(tar_vocab_size, latent_dim) #임베딩 층
dec_emb = dec_emb_layer(decoder_inputs) #패딩 0은 연산에서 제외
dec_masking = Masking(mask_value=0.0)(dec_emb)

# 상태값 리턴을 위해 return_state는 True, 모든 시점에 대해서 단어를 예측하기 위해 return_sequences는 True
decoder_LSTM = LSTM(latent_dim, return_sequences=True, return_state=True)

# 인코더의 은닉 상태를 초기 은닉 상태(initial state)로 사용
decoder_outputs, _, _ = decoder_lstm(dec_masking, initial_state=encoder_states)

# 모든 시점의 결과에 대해서 소프트맥스 함수를 사용한 출력층을 통해 단어 예측
decoder_dense = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
# 모델의 입력과 출력을 정의하므로서 모델을 정의
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

* seq2seq 디코더는 기본적으로 각각의 timestep에 대해서 다중 클래스 분류 문제를 풀고있음
  * 매 시점마다 프랑스어 단어 집합의 크기의 선택지에서 단어를 1개 선택하여 이를 이번 시점에서 예측한 단어로 택함
  * 다중 클래스 분류 문제이므로 위의 설계에서 출력층으로 소프트맥스 함수 사용
  * 이 경우, 손실 함수를 지금까지 categorical_crossentropy 사용

* categorical_crossentropy 쓰려면 **레이블은 원핫 인코딩이 된 상태**여야 함
  * 그런데 decoder_outputs의 경우에는 원핫 인코딩을 하지 않은 상태
  * **원핫 인코딩을 하지 않은 상태에서, 정수 레이블에 대해서 다중 클래스 분류 문제를 풀고자 하는 경우**에는 **categorical_crossentropy 함수가 아니라 sparse_categorical_crossentropy**를 사용하면 됨

In [ ]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [4]:
# 모델의 파라미터 확인
model.summary()

NameError: ignored

In [ ]:
# 모델 훈련
model.fit(x = [encoder_input_train, decoder_input_train], y = decoder_target_train, \
          validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size = 128, epochs = 50)

### 3. seq2seq 기계 번역기 동작시키기
* seq2seq는 훈련 과정과 테스트 과정에서의 동작 방식이 많이 달라 테스트 과정을 위해 다시 설계해야 함 esp 디코더

In [ ]:
# 인코더
encoder_model = Model(encoder_inputs, encoder_states)

* 디코더 설계

In [ ]:
# 디코더
# 이전 시점의 상태를 보관할 텐서
decoder_state_input_h = Input(shape=(latent_dim, ))
decoder_state_input_c = Input(shape=(latent_dim, ))
decoder_state_inputs = [decoder_state_input_h, decoder_state_input_c]

# 훈련 때 사용했던 임베딩 층을 재사용
dec_emb2 = dec_emb_layer(decoder_inputs)

# 다음 단어 예측을 위해 이전 시점의 상태를 현 시점의 초기 상태로 사용
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

# 모든 시점에 대해서 단어 예측
decoder_outputs2 = decoder_dense(decoder_outputs2)

In [ ]:
# 디코더를 정의
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [ ]:
# 테스트 과정에서의 동작을 위한 decode_sequence
def decode_sequence(input_seq):
  # 입력으로부터 인코더의 상태를 얻음
  states_value = encoder_model.predict(input_seq)

  # <SOS>에 해당하는 정수 생성
  target_seq = np.seros((1,1))
  target_seq[0,0] = tar_to_index['<sos']

  stop_condition = False
  decoded_sentence = ''

  # stop_condition이 True가 될 때까지 루프 반복
  # 구현의 간소화를 위해서 이 함수는 배치 크기를 1로 가짐
  while not stop_condition:
    # 이전 시점의 상태 states_value를 현 시점의 초기 상태로 사용
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

    # 예측 결과를 단어로 변환
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = index_to_tar[sampled_token_index]

    # 현재 시점의 예측 단어를 예측 문장에 추가
    decoded_sentence += ' '+sampled_char

    # <eos>에 도달하거나 정해진 길이를 넘으면 중단
    if (sampled_char == '<eos>' or 
        len(decoded_sentence) > 50):
      stop_condition = True


    # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = sampled_token_index

    # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
    states_value = [h, c]

  return decoded_sentence

In [ ]:
# 결과 확인을 위한 함수


# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2src(input_seq):
  sentence = ''
  for i in input_seq:
    if(i!=0):
      sentence = sentence + index_to_src[i] + ' '
  return sentence

# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2star(input_seq):
  sentence = ''
  for i in input_seq:
    if((i!= and i!=tar_to_index['<sos>']) and i!=tar_to_index['<eos>']):
      sentence = sentence + index_to_tar[i] + ' '
  return sentence

In [ ]:
# 훈련 데이터에 대해서 임의로 선택한 인덱스의 샘플의 결과 출력
for seq_index in [3, 50, 100, 300 ,1001]:
  input_seq = encoder_input_train[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)

  print("원문 : ",seq2src(encoder_input_train[seq_index]))
  print("번역문 :",seq2tar(decoder_input_train[seq_index]))
  print("예측문 :",decoded_sentence[1:-5])
  print("\n")

In [ ]:
# 테스트 데이터에 대해서 임의로 선택한 인덱스의 샘플의 결과 출력
for seq_index in [3,50,100,300,1001]:
  input_seq = encoder_input_test[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)

  print("원문 : ",seq2src(encoder_input_test[seq_index]))
  print("번역문 :",seq2tar(decoder_input_test[seq_index]))
  print("예측문 :",decoded_sentence[1:-5])
  print("\n")

### 3) BLEU Score(Bilingual Evaluation Understudy Score)
* 언어 모델(Language model)의 evaluation metric으로 perplexity(PPL)을 소개한 적 있음
* 논문 BLEU: a Method for Automatic Evaluation of Machine Translation를 참고

In [1]:
from collections import Counter
import numpy as np
from nltk import ngrams

#### 1. BLEU
* 기계 번역 결과와 사람이 직접 번역한 결과가 얼마나 유사한지 비교하여 번역에 대한 성능을 측정
* 측정 기준은 n-gram에 기반
* 장점: 언어에 구애받지 않고 사용 가능, 계싼 속도 빠름, PPL과는 다르게 높을수록 성능이 더 좋음을 의미
##### 1) 단어 개수 카운트로 측정하기(Unigram prediction)
* 번역된 문장 Ca 1, 2, 사람이 번역한 문장 Ref 1, 2, 3
  * Ref 1, 2, 3 중 어느 한 문장이라도 등장한 단어의 개수를 Ca에서 세는 것
  * 그 후에 Ca의 모든 단어의 카운트의 합, 즉 Ca에서의 총 단어의 수로 나눠줌
  * **유니그램 정밀도(Unigram Precision)**
##### 2) 중복을 제거하여 보정하기(Modified Unigram Precision)
  * 우선 유니그램이 하나의 Ref에서 최대 몇 번 등장했는지를 카운트
    * 이 값을 maximum reference count를 줄인 의미에서 Max_Ref_Count라고 부름. Max_Ref_Count가 기존의 단순한 카운트한 값보다 작을 경우에는 이 값을 최종 카운트 값으로 대체
  * Count clip = min(Count, Max_Ref_Count)
  * **보정된 유니그램 정밀도(Modified Unigram Precision)**이라고 부름
```
# This is formatted as code
# Candidate : the the the the the the the
# Reference1 : the cat is on the mat
# the의 경우에는 Ref1에서 총 두 번 등장하였으므로, 이제 the의 카운트는 2로 보정됩니다. Ca의 기존 유니그램 정밀도는 7/7이었으나 보정된 유니그램 정밀도는 2/7와 같이 변경됩
```
##### 5) 보정된 유니그램 정밀도 구현하기(Modified Unigram Precision)
* 유니그램을 카운트하는 Count 함수 for 분모
* Count clip 함수 두 가지 함수를 구현해야 함 for 분자


In [3]:
# 단순 카운트 함수
def simple_count(tokens, n): # 토큰화된 candidate 문장, n-gram에서의 n이 두 가지를 인자로 받음
  return Counter(ngrams(tokens, n)) # 문장에서 n-gram을 카운트

# 토큰화된 문장을 입력받아서 문장 내의 n-gram의 개수를 카운트하는 함수
# 구하고자 하는 것은 유니그램 정밀도, 그러므로 카운트하고자 하는 n-gram의 단위를 결정하는 simple_count 함수의 두번째 인자인 
# n의 값을 1로 하여 함수를 실행

In [4]:
candidate = "It is a guide to action which ensures that the military always obeys the commands of the party."
tokens = candidate.split() #단어 토큰화
result = simple_count(tokens, 1) #토큰화된 문장, 유니그램의 개수를 구하고자 한다면 n=1
print(result)

# 모든 유니그램을 카운트한 결과를 보여줌
# 대부분의 유니그램이 1개씩 카운트 되었으나 유니그램 the는 문장에서 3번 등장하였으므로 유일하게 3의 값을 가짐

Counter({('the',): 3, ('It',): 1, ('is',): 1, ('a',): 1, ('guide',): 1, ('to',): 1, ('action',): 1, ('which',): 1, ('ensures',): 1, ('that',): 1, ('military',): 1, ('always',): 1, ('obeys',): 1, ('commands',): 1, ('of',): 1, ('party.',): 1})


In [6]:
# Example 2의 Ca를 가지고 함수 수행
candidate = "the the the the the the the"
tokens = candidate.split() #단어 토큰화
result = simple_count(tokens, 1)
print(result)

# simple_count 함수는 단순 카운트를 수행하므로 the에 대해서 7이라는 카운트 값을 리턴

Counter({('the',): 7})


In [10]:
# Count clip을 아래 count_clip 이름을 가진 함수로 구현
def count_clip(candidate, reference_list, n):
  cnt_ca = simple_count(candidate, n)
  # Ca 문장에서 n-gram 카운트
  temp=dict()

  for ref in reference_list: #다수의 Ref 문장에 대해서 이하 반복
    cnt_ref = simple_count(ref, n)
    # Ref 문장에서 n-gram 카운트

    for n_gram in cnt_ref: # 모든 ref에 대해서 비교하여 특정 n-gram이 하나 Ref에 가장 많이 등장한 횟수를 저장
      if n_gram in temp:
        temp[n_gram] = max(cnt_ref[n_gram], temp[n_gram]) #max_ref_count
      else:
        temp[n_gram] = cnt_ref[n_gram]
  return {
      n_gram: min(cnt_ca.get(n_gram, 0), temp.get(n_gram, 0)) for n_gram in cnt_ca
      # count_clip=min(count, max_ref_count)
      # 위의 get은 찾고자 하는 n-gram이 없으면 0을 반환한다.
  }

# count_clip 함수는 candidate 문장과 reference 문장들, 그리고 카운트 단위가 되는 n-gra에서의 n의 값 이 세 가지를 인자로 입력받아서 count clip 수행
# Count clip을 구하기 위해서는 Max_Ref_Count값과 비교하기 위해 Count값이 필요하기 때문
# Example2를 통해 함수가 정상 작동되는지 확인해보기

In [11]:
candidate = 'the the the the the the the'
references = [
              'the cat is on the mat',
              'there is a cat on the mat'
]
result = count_clip(candidate.split(), list(map(lambda ref: ref.split(), references)), 1)
print(result)

{('the',): 2}


In [14]:
# 두 함수를 사용하여 예제 문장에 대해서 보정된 정밀도를 연산하는 함수를 modified_precision란 이름의 함수로 구현

def modified_precision(candidate, reference_list, n):
    clip = count_clip(candidate, reference_list, n) 
    total_clip = sum(clip.values()) # 분자

    ct = simple_count(candidate, n)
    total_ct = sum(ct.values()) #분모

    if total_ct==0: # n-gram의 n이 커졌을 때 분모가 0이 되는 것을 방지
      total_ct=1

    return (total_clip / total_ct) # 보정된 정밀도
    # count_clip의 합을 분자로 하고 단순 count의 합을 분모로 하면 보정된 정밀도

In [15]:
result=modified_precision(candidate.split(),list(map(lambda ref: ref.split(), references)),1) # 유니그램이므로 n=1
print(result)

0.2857142857142857


##### 4) 순서를 고려하기 위해서 n-gram으로 확장하기
* 각 단어의 빈도수로 접근하는 방법은 결국 단어의 순서를 고려하지 않는다는 특징
* Ca3은 사실 Ca1에서 모든 유니그램의 순서를 랜덤으로 섞은 실제 영어 문법에 맞지 않은 문장
* 하지만, Ref 1, 2, 3과 비교하여 유니그램 정밀도를 적용하면 Ca1과 Ca3의 두 정밀도는 동일 b/c 유니그램의 순서를 전혀 고려하지 않기에
* 이를 위한 대안으로 n-gram을 이용한 정밀도를 도입
  * 카운트 단위를 기반으로 2-gram precision, 3-gram precision, 4-gram precision이라고도 함
  * 식으로 써보면...
    ```
    pn = sum of n-grams in candidate Count clip(n-gram) / sum of n-grams in candidate Count(n-grams)
    ```
* BLEU의 식: exp(sum of n=1~n wn * log pn) 
  * pn: 각 gram의 보정된 정밀도
  * N: n-gram에서 n의 최대 숫자. 보통은 4(p1, p2, p3, p4)
  * wn: 각 gram의 보정된 정밀도에서 서로 다른 가중치를 줄 수 있음


##### 5) 짧은 문장 길이에 대한 패널티(Bervity Penalty)
* n-gram으로 단어의 순서를 고려한다고 하더라도 여전히 남아있는 문제가 있는데, 바로 Ca의 길이에 BLEU의 점수가 과한 영향을 받을 수 있다는 점
  * **브레버티 패널티(Bervity Penalty)**: 짧음 패널티
  * 문장이 짧은 경우에는 점수에 패널티를 줄 필요가 있음

* **브레버티 패널티**는 앞서 배운 BLEU의 식에 곱하는 방식으로 사용
* BLEU에 BP를 붙임
  * If c>r, BP = 1, else: c<= r, BP=e^(1/r/c)
  * c: candidate의 길이, r: candidate와 가장 길이 차이가 작은 reference의 길이

In [16]:
def closest_ref_length(candidate, reference_list): #Ca 길이와 가장 근접한 Ref의 길이를 리턴하는 함수
  ca_len = len(candidate) # ca 길이
  ref_lens = (len(ref) for ref in reference_list) #Ref들의 길이
  closest_ref_len = min(ref_lens, key=lambda ref_len: (abs(ref_len - ca_len), ref_len))
  # 길이 차이를 최소화하는 Ref를 찾아서 Ref의 길이를 리턴
  return closest_ref_len

# 만약 Ca의 길이가 정확히 동일한 Ref가 있다면 길이 차이가 0인 최고 수준의 매치(best match length)임
# 만약 서로 다른 길이의 Ref지만 Ca와 길이 차이가 동일한 경우에는 더 작은 길이의 Ref를 택함
# 예를 들어 Ca가 길이가 10인데, Ref 1, 2가 각각 9와 11이라면 길이 차이는 동일하게 1밖에 나지 않지만 9를 택함

In [17]:
def brevity_penalty(candidate, reference_list):
  ca_len = len(candidate) # ca 길이
  ref_lens = (len(ref) for ref in reference_list) # Ref들의 길이
  closest_ref_len = min(ref_lens, key=lambda re_len: (abs(ref_len - ca_len), ref_len))
  # 길이 차이를 최소화하는 Ref를 찾아서 Ref의 길이를 리턴
  return closest_ref_len

In [18]:
# 만약 Ca와 길이가 정확히 동일한 Ref가 있다면 길이 차이가 0인 최고 수준의 매치(best match length)입니다.
# 또한 만약 서로 다른 길이의 Ref이지만 Ca와 길이 차이가 동일한 경우에는 더 작은 길이의 Ref를 택함
# Ca의 길이가 10인데, Ref 1, 2가 각각 9와 11이라면 길이 차이는 동일하게 1 밖에 나지 않지만 9를 택함
# closest_ref_length 함수를 통해 r을 구했다면 이제 BP를 구하는 함수 brevity_penalty를 구현

def brevity_penalty(candidate, reference_list):
  ca_len = len(candidate)
  ref_len = closest_ref_length(candidate, reference_list)

  if ca_len > ref_len:
    return 1
  elif ca_len == 0:
  # candidate가 비어있다면 BP = 0 -> BLEU = 0.0
    return 0
  else:
    return np.exp(1 - ref_len/ca_len)

# 앞서 배운 BP 수식처럼 c가 r보다 클 경우에는 1 리턴, 그 외의 경우에는 e^((1-r)/c)를 리턴

In [19]:
# 최종적으로 BLEU 점수를 계산하는 함수 blue_score를 구현
def bleu_score(candidate, reference_list, weights=[0.15, 0.25, 0.25, 0.25]):
  bp = brevity_penalty(candidate, reference_list) #브레버티 패널티, BP

  # p1, p2, p3, ..., pn
  p_n = [modified_precision(candidate, reference_list, n=n) for n, _ in enumerate(weights, start=1)]
  score = np.sum([w_i * np.log(p_i) if p_i != 0 else 0 for w_i, p_i in zip(weights, p_n)])
  return bp * np.exp(score)

In [20]:
# 위의 blue_score 함수는 기본적으로 N이 4에 각 gram에 대한 가중치는 동일하게 0.25라 주어진다고 가정.
# 또한 함수 내에서는 BP를 구하고 bp에 p1, p2, ..., pn을 구하여 p_n에 저장하도록 구현
# 앞서 배운 BLEU의 식에 따라 추가 연산하여 최종 계산한 값을 리턴

# 위 함수가 동작하기 위해서는 앞서 구현한 simple_count, count_clip, modified_precision, brevity_penalty 4개의 함수 또한 
# 모두 구현되어져 있어야 함

#### 2. NLTK를 이용한 BLEU 측정하기

In [24]:
import nltk.translate.bleu_score as bleu

candidate = 'It is a guide to action which ensures that the military always obeys the commands of the party'
references = [
    'It is a guide to action that ensures that the military will forever heed Party commands',
    'It is the guiding principle which guarantees the military forces always being under the command of the Party',
    'It is the practical guide for the army always to heed the directions of the party'
]

# 이번 챕터에서 구현한 코드로 계산한 BLEU 점수
print(bleu_score(candidate.split(),list(map(lambda ref: ref.split(), references))))
# NLTK 패키지 구현되어져 있는 코드로 계산한 BLEU 점수
print(bleu.sentence_bleu(list(map(lambda ref: ref.split(), references)),candidate.split()))

0.5074589651750404
0.5045666840058485
